# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Chat operation
* This notebook shows you how to send chat prompts to the Gemini
You can interact with Gemini using a single-turn prompt and response or chat with it in a multi-turn, continuous conversation, even for code understanding and generation.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # To access contents in Google drive
    # from google.colab import drive
    # drive.mount('/content/drive')

## Set the environment on GCP Project


In [4]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [5]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "You are an AI Specialist",
 "Answer to the question in detail"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [6]:
from vertexai.generative_models import ChatSession

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    """
    Generate a response from the model.

    - chat : ChatSession Chat session object
    - prompt : str - The prompt to send to the model.
    - returns: str - The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = None,
        stream=False
    )

    return responses.text


## Run example

In [15]:
from time import perf_counter

def interactive_chat(chat, question:str)->str:
  """
  Interactive chat with the model.
  """

  t1_start = perf_counter()

  answer = get_chat_response(chat, question)

  display(Markdown(f"Question : {question}"))
  display(Markdown(f"Answer : {answer}"))

  t1_end  = perf_counter()
  print(f"\nElapsed Time : {t1_end - t1_start} seconds\n\n")


In [16]:

chat = model.start_chat()

history = " ".join([content.text for content in chat.history])
print(f"Chat history : {history}")

interactive_chat(chat, "Hello.")


Chat history : 


Question : Hello.

Answer : Hello! 👋  

How can I assist you today?  I'm ready to answer your questions and engage in a conversation.  What's on your mind? 😊 



Elapsed Time : 0.8927195109999957 seconds




In [17]:
history = " ".join([content.text for content in chat.history])
print(f"Chat history : {history}")

interactive_chat(chat, "What are all the colors in a rainbow?.")



Chat history : Hello. Hello! 👋  

How can I assist you today?  I'm ready to answer your questions and engage in a conversation.  What's on your mind? 😊 



Question : What are all the colors in a rainbow?.

Answer : The colors in a rainbow are often remembered with the acronym **ROYGBIV**:

* **R**ed
* **O**range
* **Y**ellow
* **G**reen
* **B**lue
* **I**ndigo
* **V**iolet

It's important to note that these colors blend seamlessly into each other, so there isn't a clear distinction between them.  The rainbow is a continuous spectrum of light! 🌈



Elapsed Time : 1.8701015039998765 seconds




In [18]:
history = " ".join([content.text for content in chat.history])
print(f"Chat history : {history}")

interactive_chat(chat, "Why does it appear when it rains?.")


Chat history : Hello. Hello! 👋  

How can I assist you today?  I'm ready to answer your questions and engage in a conversation.  What's on your mind? 😊 
 What are all the colors in a rainbow?. The colors in a rainbow are often remembered with the acronym **ROYGBIV**:

* **R**ed
* **O**range
* **Y**ellow
* **G**reen
* **B**lue
* **I**ndigo
* **V**iolet

It's important to note that these colors blend seamlessly into each other, so there isn't a clear distinction between them.  The rainbow is a continuous spectrum of light! 🌈



Question : Why does it appear when it rains?.

Answer : You're right to ask!  Rainbows are a beautiful result of sunlight and water droplets working together. Here's the breakdown:

1. **Sunlight:**  Sunlight is made up of all the colors of the rainbow, but we usually see it as white light.

2. **Water Droplets:** When rain falls, tiny water droplets are suspended in the air.

3. **Refraction and Reflection:**  As sunlight passes through a water droplet, it bends (refracts).  This bending separates the white light into its individual colors.  Then, the light reflects off the back of the droplet and bends again as it leaves. 

4. **Angle is Key:**  For us to see a rainbow, the angle between the sun, the water droplet, and our eye needs to be just right. This is why we usually see rainbows after it rains, when the sun is shining and there are still water droplets in the air.

5. **Your View:**  The rainbow you see is actually a reflection of the sunlight in the water droplets. It's like a giant, colorful reflection in the sky!

So, the next time you see a rainbow, remember that it's a beautiful display of science in action! 🌈✨



Elapsed Time : 2.122877307999943 seconds




In [19]:
# Clear chat history

chat.history.clear()

history = " ".join([content.text for content in chat.history])
print(f"Chat history : {history}")

Chat history : 


In [20]:
history = " ".join([content.text for content in chat.history])
print(f"Chat history : {history}")

interactive_chat(chat, "Why does it appear when it rains?.")

Chat history : 


Question : Why does it appear when it rains?.

Answer : Please provide more context!  "It" is very vague.  To help me understand what you're asking about, tell me:

* **What is "it" referring to?**  Is it a specific phenomenon, like a rainbow or a puddle?  Or something else?
* **What do you mean by "appear"?**  Does it become visible, change color, or something else entirely? 

Once I have more information, I can give you a detailed and accurate explanation! 



Elapsed Time : 1.939458962999879 seconds


